# 1. Download DINOv2 Git repo

In [ ]:
# Install latest compatible torch/torchvision
!pip install torch==2.4.0 torchvision==0.19.0 --quiet

# Clone and install DINOv2 code only (no torch downgrade)
!git clone https://github.com/facebookresearch/dinov2.git
%cd dinov2
!pip install . --no-deps --quiet
%cd ..

^C
d:\DINOv2_medium_article\dinov2


Cloning into 'dinov2'...
